In [1]:
from nltk.corpus import stopwords

from html import unescape

from w3lib.html import remove_tags

import pandas as pd

import re

import textrank

import importlib
# importlib.reload(textrank)

from tqdm import tqdm

tqdm.pandas()

import multiprocessing as mp

import numpy as np

%matplotlib inline
import glob
import os

import artm

import spacy

nlp = spacy.load('en_core_web_sm')

import pickle

pd.options.display.max_colwidth=300

from collections import Counter

/home/anisimova-anna/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:198: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)


In [2]:
df = pd.read_csv('data/df_hni_2018_05_orig.csv')

/home/anisimova-anna/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head(1)

,by,dead,deleted,descendants,id,kids,parent,parts,poll,retrieved_on,score,text,time,title,type,url
0,burntwater,NaN,NaN,NaN,16963601,NaN,16962078.0,NaN,NaN,1528401048,NaN,The New York State fee by itself is $200. Unle...,1525132803,NaN,comment,NaN


In [4]:
add_stops = ['ive', 'like', 'im', 'would']

url_regex = re.compile(
            r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 
            re.IGNORECASE
        )
stopwords_list = stopwords.words('english') + add_stops

def clean_text(text):
    text = text.replace('#', ' hash ')
    text = text.replace('+', ' plus ')
    text = re.sub(re.compile(r'\d+'), ' num ', text)
    text = re.sub(re.compile(r'\s+'), ' ', text)
    text = ''.join(re.findall(re.compile(r'[\w\s]+'), text))
    text = ' '.join(list(filter(lambda x: x.lower() not in stopwords_list, text.split())))
    text = text.lower()
    return text

def preprocees_text(text):
    text = remove_tags(remove_tags(unescape(text)))
    text = re.sub(url_regex, ' url ', text)
    return text

In [5]:
df.loc[:, 'text'] = df['text'].fillna('').progress_apply(preprocees_text)

100%|██████████| 237646/237646 [00:12<00:00, 19209.06it/s]


In [6]:
df.loc[:, 'clean_text'] = df['text'].fillna('').progress_apply(clean_text)

100%|██████████| 237646/237646 [00:58<00:00, 4035.70it/s]


In [7]:
df1 = df.loc[df['clean_text'].apply(lambda x: len(x.split())) >= 13, :]

8 мин -- 24тыс примеров!!!!

In [8]:
docs = list(tqdm(nlp.pipe(df1['text'].values), total=df1.shape[0]))

100%|██████████| 139301/139301 [1:00:41<00:00, 38.26it/s]


In [11]:
ents = list(map(lambda x: Counter([ent.label_ for ent in x.ents]), docs))

with open('./data/preprocessed/spacy_ents_more_13.pkl', 'wb') as f:
    pickle.dump(ents, f)

In [15]:
df1.loc[:, 'ents'] = np.array(ents, dtype=object)

/home/anisimova-anna/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/anisimova-anna/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [17]:
df1.loc[:, 'ents_modal'] = df1['ents']\
    .progress_apply(lambda x: ' '.join(['{}:{}'.format(a, b) for a,b in x.items()]))

100%|██████████| 139301/139301 [00:00<00:00, 252683.84it/s]


pool = mp.Pool(mp.cpu_count() - 1)
result = pool.map(textrank.X2_test,
                  df1['clean_text'].values)

with open('./data/preprocessed/textrank_kw_more_13.pkl', 'wb') as f:
    pickle.dump(result, f)

In [19]:
with open('./data/preprocessed/textrank_kw_more_13.pkl', 'rb') as f:
    result_kw = pickle.load(f)

In [20]:
df1.loc[:, 'textrank'] = np.array(result_kw, dtype=object)

In [22]:
df1['textrank'] = df1['textrank'].apply(lambda x: ' '.join(x))

df1 = df1.reset_index(drop=True)

df1.reset_index(drop=True).to_csv('./data/preprocessed/df_textrank_clean_more_13.csv', index=None)

/home/anisimova-anna/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


TOPMINE

In [24]:
with open('../_work_bigARTM_logs/_bigARTM_research/TopMine/topmine/input/for_topmine.txt', 'w') as f:
    for row in df['clean_text'].values:
        f.write(row)
        f.write('./n')

In [28]:
with open('../_work_bigARTM_logs/_bigARTM_research/TopMine/topmine/output/frequent_phrases.txt', 'r') as f:
    f_w = f.readlines()

In [34]:
phrases = [eval(re.sub(re.compile(r'\d+ '), '', phrase)) for phrase in f_w]

In [36]:
cnt_ph = 30

top_n_phs = list(filter(lambda x: (x[1] > cnt_ph) and ('num' not in x[0]), phrases))
top_n_phs = [' {} '.format(x[0]) for x in top_n_phs]

In [38]:
def replace_or_add_phs(text):
    text = ' {} '.format(text)
    regime='replace'
    def find_top_phs(top_n_phs, text):
        res = []
        for ph in top_n_phs:
            if ph in text:
                res.append(ph)
        return res
    small_ph_list = find_top_phs(top_n_phs, text)
    final_ph_list = list(map(lambda x: ' {} '.format('_'.join(x.strip().split())), small_ph_list))
    if regime=='add':
        text += ' ' + ' '.join(final_ph_list)
    else:
        for phrase, replace in zip(small_ph_list, final_ph_list):
            text = text.replace(phrase, replace)
    return text

In [42]:
pool4 = mp.Pool(mp.cpu_count() - 1)
result_ngram = pool4.map(replace_or_add_phs,
                  df1['clean_text'].values)

In [43]:
df1.loc[:, 'replace_ngram'] = np.array(result_ngram, dtype=object)

In [46]:
df1.to_csv('./data/preprocessed/df_textrank_clean_replace_more_13.csv', index=None)

In [47]:
df1.head(1)

,by,dead,deleted,descendants,id,kids,parent,parts,poll,retrieved_on,...,text,time,title,type,url,clean_text,ents,ents_modal,textrank,replace_ngram
0,burntwater,NaN,NaN,NaN,16963601,NaN,16962078.0,NaN,NaN,1528401048,...,The New York State fee by itself is $200. Unle...,1525132803,NaN,comment,NaN,new york state fee num unless missing somethin...,"{'GPE': 1, 'MONEY': 2, 'CARDINAL': 1}",GPE:1 MONEY:2 CARDINAL:1,fee requirement take significantly hundred giv...,new_york state fee num unless missing_somethi...


In [ ]:
def post_to_wabbit_multimodals(i, text, textrank):
    text_cnt = ' '.join(['{}:{}'.format(x, y) for x,y in Counter(text.split()).items()])
    row = '{} |@default_class {} |@textrank {}'\
        .format(i, text_cnt, textrank)
    row = re.sub(r'\s+', ' ', row)
    return row

In [48]:
def post_to_wabbit_multimodals(i, text, textrank, ents):
    text_cnt = ' '.join(['{}:{}'.format(x, y) for x,y in Counter(text.split()).items()])
    row = '{} |@default_class {} |@textrank {} |@ents {}'\
        .format(i, text_cnt, textrank, ents)
    row = re.sub(r'\s+', ' ', row)
    return row

In [49]:
df1.columns

Index(['by', 'dead', 'deleted', 'descendants', 'id', 'kids', 'parent', 'parts',
       'poll', 'retrieved_on', 'score', 'text', 'time', 'title', 'type', 'url',
       'clean_text', 'ents', 'ents_modal', 'textrank', 'replace_ngram'],
      dtype='object')

In [50]:
def creacte_dict_multimodal(df, name, column):
    
    batch_folder = './{}_batches'.format(name)
    data_path = './{}_vw.txt'.format(name)
    dict_path = batch_folder + '/dictionary.dict'  
    
    dic = artm.Dictionary()
    
    if os.path.isfile(dict_path):
        dic.load(dictionary_path=dict_path)
        
        batch_vectorizer = artm.BatchVectorizer(data_path=data_path, 
                                        data_format='vowpal_wabbit', 
                                        collection_name='nthhappens', 
                                        target_folder=batch_folder)
        return batch_vectorizer, dic
    
    with open('./{}_vw.txt'.format(name), 'w', encoding='utf8') as f:
        for i, row in df.iterrows():
            line = post_to_wabbit_multimodals(i, row[column], row['textrank'], row['ents_modal'])
            f.write(line)
            f.write('\n')        
    
    batch_vectorizer = artm.BatchVectorizer(data_path=data_path, 
                                        data_format='vowpal_wabbit', 
                                        collection_name='nthhappens', 
                                        target_folder=batch_folder)
    
    
    dic.gather(data_path=batch_vectorizer.data_path)
    dic.save(dictionary_path=dict_path)
    
    return batch_vectorizer, dic

In [52]:
batch_vectorizer, dic = creacte_dict_multimodal(df=df1, 
                                                name='clean_replace_ent_text_textrank_more_13', 
                                                column='replace_ngram')